## Part 1: Preprocessing

In [23]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [24]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [25]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [26]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 'DistanceFromHome', 'Education', 'JobLevel', 'EnvironmentSatisfaction', 
             'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel', 
             'TotalWorkingYears', 'YearsAtCompany']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes

Age                        int64
DistanceFromHome           int64
Education                  int64
JobLevel                   int64
EnvironmentSatisfaction    int64
NumCompaniesWorked         int64
PercentSalaryHike          int64
StockOptionLevel           int64
TotalWorkingYears          int64
YearsAtCompany             int64
dtype: object

In [27]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [ ]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary


,count
OverTime,
No,797
Yes,305


In [28]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)

In [29]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
encoder_department = OneHotEncoder(sparse_output=False)


# Fit the encoder to the training data
y_department = encoder_department.fit_transform(y_df[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data


In [30]:
# Create a OneHotEncoder for the Attrition column
encoder_attrition = OneHotEncoder(sparse_output=False)


# Fit the encoder to the training data
y_attrition = encoder_attrition.fit_transform(y_df[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data

In [31]:
#train_test_split on new variables
X_train, X_test, y_train_attrition, y_test_attrition, y_train_department, y_test_department = train_test_split(X_df, y_attrition, y_department, random_state=1)

## Part 2: Create, Compile, and Train the Model

In [32]:
# Find the number of columns in the X training data.
len(X_train_scaled[0])

# Create the input layer
input_layer = Input(shape=(X_train_scaled.shape[1],))

# Create at least two shared layers
shared_layer_1 = Dense(64, activation='relu')(input_layer)
shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)

In [33]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
department_output = Dense(y_train_department.shape[1], activation='softmax', name='department_output')(department_hidden)

In [34]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
attrition_output = Dense(y_train_attrition.shape[1], activation='softmax', name='attrition_output')(attrition_hidden)

In [37]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'categorical_crossentropy'},
    metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'}
)

# Summarize the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        704 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         51 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         34 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,925 (15.33 KB)

 Trainable params: 3,925 (15.33 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
# Train the model
history = model.fit(
    X_train_scaled, {'department_output': y_train_department, 'attrition_output': y_train_attrition},
    validation_data=(X_test_scaled, {'department_output': y_test_department, 'attrition_output': y_test_attrition}),
    epochs=100,
    batch_size=32
)


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.9416 - attrition_output_loss: 0.1753 - department_output_accuracy: 0.7966 - department_output_loss: 0.4939 - loss: 0.6688 - val_attrition_output_accuracy: 0.7962 - val_attrition_output_loss: 0.6086 - val_department_output_accuracy: 0.6141 - val_department_output_loss: 0.8803 - val_loss: 1.5191
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.9215 - attrition_output_loss: 0.1884 - department_output_accuracy: 0.7936 - department_output_loss: 0.5017 - loss: 0.6901 - val_attrition_output_accuracy: 0.7962 - val_attrition_output_loss: 0.6206 - val_department_output_accuracy: 0.6277 - val_department_output_loss: 0.8985 - val_loss: 1.5540
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.9173 - attrition_output_loss: 0.1988 - department_output_accuracy: 0.7959 - department_output_loss: 0.4932 - loss: 0.6919 - val_attrition_output_accuracy: 0.7908 - 

In [40]:
# Evaluate the model with the testing data
eval_results = model.evaluate(X_test_scaled, {'department_output': y_test_department, 'attrition_output': y_test_attrition})

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.7860 - attrition_output_loss: 1.6630 - department_output_accuracy: 0.5660 - department_output_loss: 1.9130 - loss: 3.5856 


In [41]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {eval_results[3]:.4f}")
print(f"Attrition Accuracy: {eval_results[4]:.4f}")

Department Accuracy: 0.7636
Attrition Accuracy: 0.5870


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. For a balanced dataset, accuracy is the most useful, but when imbalanced, other metrics such as precision and recall could be more useful.
2. I chose Softmax both department and attrition were categorical variables with multiple possible values.
3.  
- Adding additional features instead of just 10.
- Use dropout layers to prevent overfitting